In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
from random import choices

In [2]:
## Calculate win or not win
## Parameter: MyStract ndarray, yourStrat ndarray
## Return win 1 or lose 0
def getScore(myStrat, yourStrat):
    myScore = 0
    yourScore = 0
    i = 0 
    while myScore < 20 and yourScore < 20 and i<10:
        if (myStrat[i] > yourStrat[i]):
            myScore += (i+1);
        if myStrat[i] < yourStrat[i]:
            yourScore += (i+1);
        i+=1
    if myScore >= 20:
        return 1
    if yourScore >= 20:
        return 0
    else:
        return 0

    
## Generate n random dirichlet
## Parameter: n strategies
## Return 2D np array
def makeStrat(alphas,n):
    strats = np.random.dirichlet(alphas,n)
    strats *= 100
    strats = np.rint(strats)
    adjust = 100 - np.sum(strats,axis=1)
    for i in range(n):
        strats[i,0] += adjust[i]
        np.random.shuffle(strats[i])
    return strats

## Generate n particular dirichlet with specified alpha
## Parameter: n strategies
## Return 2D np array
def SampleIntDir(alphas,n):
    strats = np.random.dirichlet(alphas,n)
    strats *= 100
    strats = np.rint(strats)
    adjust = 100 - np.sum(strats,axis=1)
    for i in range(n):
        dim = np.random.randint(0,10)
        strats[i,dim] += adjust[i]
    return strats

## Calculate average win
## Parameter: MyStrat ndarray, testing ndarrays
## Return winning rate
def getMultiScore(myStrat, yourStrats):
    sum = 0
    for i in range(yourStrats.shape[0]):
        sum +=getScore(myStrat, yourStrats[i])
    return sum/yourStrats.shape[0]

## Calculate average win
## Parameter: MyStrat ndarrays, testing ndarrays
## Return winning rate
def getMultiMultiScore(myStrats, yourStrats): 
    sumsum = 0
    for j in range(myStrats.shape[0]):
        sum = 0
        for i in range(yourStrats.shape[0]):
            sum +=getScore(myStrats[j], yourStrats[i])
    sumsum += sum/yourStrats.shape[0]
    return sumsum

## Get a neighbourhood o
def getNeighbourhood(x,n):
    xs = []
    for i in range(n):
        noise = np.random.randint(-2,2,size=9)
        y = x + np.append(noise,-np.sum(noise))
        if np.all(np.greater_equal(y,np.zeros(10))) and np.all(np.less_equal(y,np.ones(10)*100)):
            xs.append(y)
    return np.array(xs)

##
def update(x, ref):
    max = 0
    win = 0
    for i in range(x.shape[0]):
        if getMultiScore(x[i],ref) > win:
            max = x[i]
            win = getMultiScore(x[i],ref)
    return (max,win)

def getSingleRef(strat,dic,S5data):
    if strat != 'S5':
        return SampleIntDir(dic[strat],1)[0]
    else:
        i = np.random.randint(S5data.shape[0])
        return S5data[i]
    

In [ ]:
"""
This is for the last Monte Carlo find best solution
"""

def main():
    
    S5=pd.read_csv('computationalApproachCentroids.csv', sep=',',header=None).values
    StratsInfo = {'S0': [1,1,1,1,1,1,1,1,1,1], 
              'S11': [1,1,1,1,100,1,100,100,1,1],
              'S12': [1,1,1,1,1,100,100,100,1,1],
              'S2': [5,5,10,15,100,100,1,10,10,1], 
              'S3': [1,10,1,15,1,1,100,50,10,1], 
              'S4': [2,4,7,9,11,13,15,18,21,1],
              'S5': S5}
    Strats = ["S0","S11","S12","S2","S3","S4","S5"]
    weight = [0.02,0.336,0.144,0.16,0.16,0.03,0.15]
    
    maxIter = 150
    numInitial = 500
    refSize = 5000
    stopThre = 4
    neighbourSize = 500
    mpl.style.use("seaborn")
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.set_ylim([0, 1])
    ax.set_title('Final Solution converge for Simulated Annealing'.format("seaborn"), color='C0')   
    
    
    x = makeStrat([1,1,1,1,1,1,1,1,1,1],numInitial)
    strats = []
    wins = []
    for j in range(numInitial):
        converge = False
        good = False
        print("Processing",j+1,"initial point:")
        centroid = x[j]
        win = []
        for i in range(maxIter):
            if i% 50 == 0:
                print("   iteration:",i)
            neigh = getNeighbourhood(centroid, neighbourSize)

            
            ##This part get ref
            ref = []
            for i in range(refSize):
                ref.append(getSingleRef(choices(Strats, weight)[0],StratsInfo,S5))
            ref =np.array(ref)           
            
            inter = update(neigh,ref)

            if np.sum(np.abs(centroid- inter[0])) < stopThre:
                print("   Converged!!")
                converge = True
                break

            centroid = inter[0]
            win.append(inter[1])
            
        ## Reject or Accept sample solution
        if win[-1] >= 0.85:
            good = True
            
        if good and converge:
        ## Reject or Accept sample Solution
            strats.append(centroid)
            pd.DataFrame(np.array(strats)).to_csv("FinalSolutions.csv",header=None, index=None)

        ## Make or not make solution plot
            win = np.concatenate((np.array(win),np.ones(maxIter-len(win))*win[-1]), axis = 0)
            wins.append(win)
            ax.plot([i+1 for i in range(len(win))],  win)    
            
    ax.plot([i+1 for i in range(len(win))],  [0.85 for i in range(len(win))], linestyle = '--', color = "yellow")
        
        
        
    plt.show() 
    fig.savefig('FinalSolutionConvergence.jpg')
        
if __name__== "__main__":
    main()

Processing 1 initial point:
   iteration: 0
   iteration: 50
   iteration: 100
   Converged!!
Processing 2 initial point:
   iteration: 0
   iteration: 50
   Converged!!
Processing 3 initial point:
   iteration: 0
   Converged!!
Processing 4 initial point:
   iteration: 0
   Converged!!
Processing 5 initial point:
   iteration: 0
   iteration: 50
   iteration: 100
   Converged!!
Processing 6 initial point:
   iteration: 0
   iteration: 50
   Converged!!
Processing 7 initial point:
   iteration: 0
   iteration: 50
   iteration: 100
   Converged!!
Processing 8 initial point:
   iteration: 0
   Converged!!
Processing 9 initial point:
   iteration: 0
   Converged!!
Processing 10 initial point:
   iteration: 0
   Converged!!
Processing 11 initial point:
   iteration: 0
   Converged!!
Processing 12 initial point:
   iteration: 0
   Converged!!
Processing 13 initial point:
   iteration: 0
   Converged!!
Processing 14 initial point:
   iteration: 0
   Converged!!
Processing 15 initial point:
 

   iteration: 100
   Converged!!
Processing 119 initial point:
   iteration: 0
   Converged!!
Processing 120 initial point:
   iteration: 0
   iteration: 50
   iteration: 100
   Converged!!
Processing 121 initial point:
   iteration: 0
   Converged!!
Processing 122 initial point:
   iteration: 0
   iteration: 50
   Converged!!
Processing 123 initial point:
   iteration: 0
   Converged!!
Processing 124 initial point:
   iteration: 0
   Converged!!
Processing 125 initial point:
   iteration: 0
   iteration: 50
   Converged!!
Processing 126 initial point:
   iteration: 0
   Converged!!
Processing 127 initial point:
   iteration: 0
   iteration: 50
   Converged!!
Processing 128 initial point:
   iteration: 0
   Converged!!
Processing 129 initial point:
   iteration: 0
   Converged!!
Processing 130 initial point:
   iteration: 0
   iteration: 50
   Converged!!
Processing 131 initial point:
   iteration: 0
   Converged!!
Processing 132 initial point:
   iteration: 0
   Converged!!
Processing

In [ ]:
"""
This is for deriving each S1,S2,S3,S4 and find the alphas
"""

def main():
    
    #for i in [14,15,16,17]:

    test = SampleIntDir([10,10,10,10,10,10,10,10,10,10],1)
    centroid = test[0]
    for i in range(1000):
        lol1 =SampleIntDir([1,1,1,1,100,1,100,100,1,1],700)
        lol2 =SampleIntDir([1,1,1,1,1,100,100,100,1,1],300)
        lol = np.concatenate((lol1,lol2),axis=0)
        random = makeStrat([1,1,1,1,1,1,1,1,1,1],300)
        ref1 = np.concatenate((lol,random),axis=0)
        beater = SampleIntDir([5,5,10,15,100,100,1,10,10,1],300)
        ref2 = np.concatenate((beater,ref1),axis=0)
        innovator = SampleIntDir([1,10,1,15, 1, 1, 100, 50, 10,1],400)
        ref  = np.concatenate((beater,ref2),axis=0)
        
        
        
        neigh = getNeighbourhood(centroid, 500)
        inter = update(neigh,ref)
        centroid = inter[0]
        
        if i&5 ==0:
            print("processing", i+1,"point")
            print("win rate=",inter[1])
            print(centroid)
        
    print("The score of this sampling dist is",
                  getMultiMultiScore(test, ref))
    
    
if __name__== "__main__":
    main()

In [ ]:
"""
This is for Computationist approach clustering and analyzing the solutions
"""

def main():
    strats=pd.read_csv('computationalApproachCentroids.csv', sep=',',header=None)
    
    kmeans = AgglomerativeClustering(3).fit(strats)
    colors = ["black","blue","red","yellow","green"]
    col = [colors[kmeans.labels_[i]] for i in range(len(strats))]
    X_embedded = PCA(n_components=2).fit_transform(strats)
    plt.scatter(X_embedded[:,0], X_embedded[:,1], color = col) 
    
    strats=pd.DataFrame(data=strats.values, columns = ["castle 1","castle 2","castle 3",
                            "castle 4","castle 5","castle 6",
                            "castle 7","castle 8","castle 9","castle 10"])
    g = sns.clustermap(strats,metric="cityblock")
    
    
if __name__== "__main__":
    main()

In [ ]:
"""
This is for Computationist approach find the best solutions
"""

def main():
    myStrat = np.array([10,10,10,20,20,27,1,1,1,1])
    centroid = myStrat
    
    
    maxIter = 80
    numInitial = 1000
    refSize = 500
    stopThre = 5
    neighbourSize = 500
    mpl.style.use("seaborn")
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.set_ylim([0, 1])
    ax.set_title('Truncated Simulated Annealing with Metrapolis Hasting'.format("seaborn"), color='C0')   

    
    x = makeStrat([1,1,1,1,1,1,1,1,1,1],numInitial)
    strats = []
    wins = []
    for j in range(numInitial):
        converge = False
        good = False
        print("Processing",j+1,"initial point:")
        centroid = x[j]
        win = []
        for i in range(maxIter):
            if i% 50 == 0:
                print("   iteration:",i)
            neigh = getNeighbourhood(centroid, neighbourSize)
            ref = makeStrat([1,1,1,1,1,1,1,1,1,1],refSize)
            inter = update(neigh,ref)

            ## Convergence check
            if np.sum(np.abs(centroid- inter[0])) < stopThre:
                print("   Converged!!")
                converge = True
                break
                
            centroid = inter[0]
            win.append(inter[1])
        
        ## Reject or Accept sample solution
        if win[-1] >= 0.85:
            good = True
            
        if good and converge:
        ## Reject or Accept sample Solution
            strats.append(centroid)
            pd.DataFrame(np.array(strats)).to_csv("computationalApproachCentroids.csv",header=None, index=None)

        ## Make or not make solution plot
            win = np.concatenate((np.array(win),np.ones(maxIter-len(win))*win[-1]), axis = 0)
            wins.append(win)
            ax.plot([i+1 for i in range(len(win))],  win)
        
    ## Threshold line
    ax.plot([i+1 for i in range(len(win))],  [0.85 for i in range(len(win))], linestyle = '--', color = "yellow")
        
    ## Show the plot, save the plot
    plt.show() 
    fig.savefig('computationalApproach.jpg')
        
if __name__== "__main__":
    main()


